In [65]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date


## Loading in Data from Invisible Institute

All data is pulled from the invisible institute github (https://github.com/invinst/chicago-police-data), and saved locally with the original file names. 


# Cleaning Department Roster

Limit officers to those on the CPD Roster in 2016. For officers that appear in multiple positions within 2016, limit to the last known position. Finally, categorize departments by overarching department clusters

In [66]:
history_roster=pd.read_csv("raw_data/roster_1936-2017_2017-04.csv")


In [67]:
df=history_roster 
df["resignation_date"]=df["resignation_date"].fillna("2999-01-01")
df["resignation_year"]=df["resignation_date"].apply(lambda x: int(str(x)[0:4]))

df["appointed_date"]=df["appointed_date"].fillna("1000-01-01")
df["appointed_year"]=df["appointed_date"].apply(lambda x: int(str(x)[0:4]))

df=df[df["appointed_year"]<2017] #officer in 2016
df=df[df["resignation_year"]>2015] #officer in 2016

#check that no one is duplicated
temp=df.UID.value_counts().reset_index()
problem_uids=temp[temp.UID>1]["index"]
df=df[~(df["UID"].isin(problem_uids) & ~df["current_status"]>0)]


In [68]:
max(df.appointed_year)

2016

In [69]:
df["unit_cat"]=np.where(df["unit_description"].isna(),"OTHER",
    np.where(df["unit_description"].str.contains("DISTRICT 0"),"DISTRICT",
    np.where(df["unit_description"].str.contains("TRAINING"),"TRAINING",
    np.where(df["unit_description"].str.contains("DETECTIVE"),"DETECTIVE",
    np.where(df["unit_description"].str.contains("SPECIAL INVESTIGATIONS UNIT"),"DETECTIVE",
    np.where(df["unit_description"].str.contains("TRAFFIC"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("TRANSPORT"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("TRANSIT"),"TRANSPORT",    
    np.where(df["unit_description"].str.contains("AIRPORT"),"TRANSPORT",
    np.where(df["unit_description"].str.contains("GANG"),"GANG ENFORCEMENT",
    "OTHER"
    ))))))))))




# Section of Complaints

Merge all complaint data on unique id ("cr_id"). Remove non-severe departmental complaints, and bucket civilian complaint types. (Note: these decissions were made mannually by reading through all of the complaint descriptions). Group the complaints by complaint type to ensure that all complaints are categorized, and none are included in multiple categories. (Note: double checking code is included but commented out). Group complaints by officer ("UID") to create complaint counts per person.  


In [70]:
complaints=pd.read_csv("raw_data/complaints-accused.csv")
complaints_detailed=pd.read_csv("raw_data/complaints-complaints.csv")
complaint_types=pd.read_csv("raw_data/Complaint Categories.csv")

cmp= complaints.merge(complaints_detailed[["cr_id", "complaint_date"]],on="cr_id",how="left")
cmp["code"]=cmp["complaint_code"]
complaint_types=complaint_types[~pd.isna(complaint_types["111"])]
complaint_types["code"]=complaint_types["111"]
cmp=cmp.merge(complaint_types,on="code",how="left")

C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_29252\3966214400.py:1: DtypeWarning: Columns (3,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints=pd.read_csv("raw_data/complaints-accused.csv")
C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_29252\3966214400.py:2: DtypeWarning: Columns (0,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_detailed=pd.read_csv("raw_data/complaints-complaints.csv")


In [71]:
dept_exclude=["OPERATION/PERSONNEL VIOLATION:  MISCELLANEOUS","MISCELLANEOUS", "OPERATION/PERSONNEL VIOLATION:  REPORTS - FAIL TO SUBMIT/IMPROPER", "OPERATION/PERSONNEL VIOLATION:  INVENTORY PROCEDURES","REPORTS - FAILED TO SUBMIT/IMPROPER","INVENTORY PROCEDURES","OPERATION/PERSONNEL VIOLATION: COMMUNICATION OPERATION PROCEDURES","COMMUNICATION OPERATIONS PROCEDURES","OPERATION/PERSONNEL VIOLATION:  VEHICLE LICENSING - CITY","MISUSE OF DEPARTMENT EQUIPMENT/SUPPLIES","INSUBORDINATION","OPERATION/PERSONNEL VIOLATION:  INSUBORDINATION","MEDICAL ROLL","OPERATION/PERSONNEL VIOLATION:  MEDICAL ROLL","OPERATION/PERSONNEL VIOLATION:  WEAPON/AMMUN./UNIFORM DEVIATION","FAIL TO OBTAIN A COMPLAINT REGISTER NUMBER","OPERATION/PERSONNEL VIOLATION: MISUSE OF DEPT. EQUIPMENT/SUPPLIES","OPERATION/PERSONNEL VIOLATION:  ABSENT WITHOUT PERMISSION","WEAPON/AMMUNITION/UNIFORM DEVIATION","SUPERVISOR RESPONSIBILITY:  FAIL TO OBTAIN COMPLAINT REGISTER NO.","SUPERVISOR RESPONSIBILITY:  MISCELLANEOUS","OPERATION/PERSONNEL VIOLATION:  LEAVING ASGMT (DIST/BEAT/SECT/CT)","ABSENT WITHOUT PERMISSION","OPERATION/PERSONNEL VIOLATION:  RESIDENCY","LEAVING ASSIGNMENT (DISTRICT, BEAT, SECTOR, COURT)","RESIDENCY","OPERATION/PERSONNEL VIOLATION:  COMPENSATORY TIME","VEHICLE LICENSING - CITY","OPERATION/PERSONNEL VIOLATION:  LATE FOR ROLL CALL/ASSIGNMENT/CT.","COMPENSATORY TIME","SUPVR. RESPONSIBILITY:  PROPER ACTION REVIEW/INSPECT- SUBORDINATE","OPERATION/PERSONNEL VIOLATION:  LUNCH/PERSONAL","LATE - ROLL CALL/ASSIGNMENT/COURT","LUNCH/PERSONAL VIOLATIONS","PROPER ACTION REVIEW/INSPECT - SUBORDINATE","EQUIPMENT IRREGULARITY"]

force=["EXCESSIVE FORCE","STRIKE","ASSAULT","BATTERY","PUSH","PULL","GRAB","CHOKED","KICKED","DRAGGED","TASER","SHOTS FIRED","COERCION - VIOLENCE","MURDER","MANSLAUGHTER","TAKE DOWN","INJURY","DEATH","FIREARM","WEAPON","PHYSICAL CONTACT"]
detain=["DETENTION","WARRANT","SEARCH","ARRESTEE","ARREST","LOCKUP","BONDING","BOOKING","PROCESSING","PRISONER'S PROPERTY", "PRISONERS PROPERTY"]
hate=["RACIAL","SEXUAL ORIENTATION","VERBAL ABUSE", "USE OF PROFANITY", "THREATS"]
crime=["COMMISSION OF CRIME", "DAMAGE","TRESPASSING", "BURGLARY", "THEFT", "FELONY","DRUGS","CONTR. SUB", "ROBBERY", "SHOPLIFTING","DOMESTIC","RAPE"]


In [72]:
cmp["civ_complaint_force"]=False
cmp["civ_complaint_detain"]=False
cmp["civ_complaint_hate"]=False
cmp["civ_complaint_crime"]=False

cmp["civ_complaint_count"]=np.where(cmp["CITIZEN / DEPT"]=="CITIZEN",1,0)
cmp["civ_complaint_count_desc"]=np.where(cmp["civ_complaint_count"],cmp.complaint_category,"")
cmp["civ_complaint_count_desc"]=np.where(cmp["civ_complaint_count_desc"].isna(),"Missing",cmp["civ_complaint_count_desc"])

for word in force:
    cmp["civ_complaint_force"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_force"])
for word in detain:
    cmp["civ_complaint_detain"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_detain"])
for word in hate:
    cmp["civ_complaint_hate"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_hate"])
for word in crime:
    cmp["civ_complaint_crime"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),True,cmp["civ_complaint_crime"])

duplicate_words_detain=["EXCESSIVE FORCE:  ARRESTEE","EXCESSIVE FORCE:  NO ARREST"]
duplicate_words_force=["COMMISSION OF CRIME:  ASSAULT/BATTERY","COMMISSION OF CRIME:  MURDER/MANSLAUGHTER ETC","ARREST/LOCKUP PROCEDURE:  PROPER CARE/INJURY/DEATH"]
duplicate_words_crime=["EXCESSIVE FORCE:  DOMESTIC"]

#this is code to identify duplicate words 
#cmp[["civ_complaint_count","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]].value_counts()
#cmp.loc[(cmp["civ_complaint_force"]==True) & (cmp["civ_complaint_detain"]==True)]["civ_complaint_count_desc"].value_counts()

for word in duplicate_words_force:
    cmp["civ_complaint_force"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),False,cmp["civ_complaint_force"])
for word in duplicate_words_detain:
    cmp["civ_complaint_detain"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),False,cmp["civ_complaint_detain"])
for word in duplicate_words_crime:
    cmp["civ_complaint_crime"]=np.where(cmp["civ_complaint_count_desc"].str.contains(word,case=False),False,cmp["civ_complaint_crime"])

cmp["dept_complaint_count"]=np.where(cmp["CITIZEN / DEPT"]=="DEPT",True,False)
cmp["dept_complaint_cleaned_count"]=np.where(cmp["CITIZEN / DEPT"]=="DEPT",np.where(cmp.complaint_category.isin(dept_exclude),False,True),False)
cmp["dept_complaint_cleaned_count_desc"]=np.where(cmp["dept_complaint_cleaned_count"],cmp.complaint_category,"")

cmp["civ_complaint_cat"]=np.where(cmp.civ_complaint_force,"FORCE",
    np.where(cmp.civ_complaint_detain,"DETAIN",
    np.where(cmp.civ_complaint_hate,"HATE",
    np.where(cmp.civ_complaint_crime,"CRIME",
    np.where(cmp.civ_complaint_count,"OTHER","")))))
    
#checking count made correctly
#cmp[cmp.dept_complaint_count==1][["complaint_category","dept_complaint_cleaned_count"]].value_counts()

In [73]:
cmp_counts=cmp.groupby("UID",as_index=False).agg({"dept_complaint_count":"sum","dept_complaint_cleaned_count":"sum","civ_complaint_count":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum"})
df=df.merge(cmp_counts,on="UID",how="left")

counts=["dept_complaint_cleaned_count","civ_complaint_count","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]
col_names=["dept_complaint_cleaned_count_desc","civ_complaint_count_desc","civ_complaint_force_desc","civ_complaint_detain_desc","civ_complaint_hate_desc","civ_complaint_crime_desc"]
for i, count in enumerate(counts):
    temp=cmp[cmp[count]==True].groupby("UID")["complaint_category"].apply(list).reset_index()
    temp.columns=["UID",col_names[i]]
    df=df.merge(temp,on="UID",how="left")

## Tactical Response Reports

Merge all tactical response reports on unique id ("trr_id"). Bucket report incidents by action type. (Note: all use of force actions were already labeled as "Physical Force" in "force_type" variable. Weapon usage was categorized mannually based on the "force_type" variable. Display of a weapon is not considered weapon usage. All other incidents are categorized as other, and not used in the analysis).  Group incidents by officer ("UID") to create incident counts per person. 


In [74]:
trr_overall=pd.read_csv("raw_data/TRR-main_2004-2016_2016-09.csv")
trr_subjects=pd.read_csv("raw_data/TRR-subjects_2004-2016_2016-09.csv")
trr_officers=pd.read_csv("raw_data/TRR-officers_2004-2016_2016-09.csv")
trr_actions=pd.read_csv("raw_data/TRR-actions-responses_2004-2016_2016-09.csv")

trr=trr_overall.merge(trr_officers,on="trr_id",suffixes=("","_officer")) #,how="left"
trr=trr.merge(trr_subjects,on="trr_id",suffixes=("","_subject")) #,how="left"
trr=trr.merge(trr_actions[trr_actions.person=="Member Action"],on="trr_id",suffixes=("","_subject"))

trr["trr_year"]=trr["trr_date"].apply(lambda x: int(str(x)[0:4]))


In [75]:
action_cat={"Missing":"Missing","Verbal Commands":"Other","Member Presence":"Other","Physical Force - Stunning":"Use of Force","Physical Force - Holding":"Use of Force","Physical Force - Direct Mechanical":"Use of Force","Other Force":"Use of Force","Taser":"Use of Weapon","Chemical":"Use of Weapon","Impact Weapon":"Use of Weapon","Firearm":"Use of Weapon","Taser Display":"Other","Chemical (Authorized)":"Use of Weapon","Other":"Other"} 

In [76]:
trr["force_cat"]=np.where(trr.force_type.isna(),"Missing",trr.force_type.replace(action_cat))
trr["count"]=True
trr["weapon_count"]=np.where(trr["force_cat"]=="Use of Weapon",True,False)
trr["force_count"]=np.where(trr["force_cat"]=="Use of Force",True,trr["weapon_count"])
trr["trr_count"]=np.where(trr["force_cat"]=="Other",True,trr["force_count"])

In [77]:
trr_counts=trr.groupby("UID",as_index=False).agg({"force_count":"sum","weapon_count":"sum","trr_count":"sum"})
    
df=df.merge(trr_counts,on="UID",how="left")

counts=["force_count","weapon_count","trr_count"]
col_names=["force_count_desc","weapon_count_desc","trr_count_desc"]
for i, count in enumerate(counts):
    temp=trr[trr[count]==True].groupby("UID")["action"].apply(list).reset_index()
    temp.columns=["UID",col_names[i]]
    df=df.merge(temp,on="UID",how="left")


# Combine and Export

Reformat variables for export: fix percentage, combine full name, calculate years on force, group complaint and self-reported incident descriptions per person. 

Group by department and export as departments.json
Group all and export as total_force.json

In [78]:
col_names=["force_count","weapon_count","trr_count","dept_complaint_count","dept_complaint_cleaned_count","civ_complaint_count","civ_complaint_force","civ_complaint_detain","civ_complaint_hate","civ_complaint_crime"]

for col in col_names:
    df[col+"_persons_with"]=np.where(df[col]>0,1,0)

In [79]:
df["one"]=1
unit_all=df.groupby("unit_description").agg({"UID":"count","force_count":"sum","weapon_count":"sum","trr_count":"sum","dept_complaint_count":"sum","dept_complaint_cleaned_count":"sum","civ_complaint_count":"sum","civ_complaint_force":"sum","civ_complaint_detain":"sum","civ_complaint_hate":"sum","civ_complaint_crime":"sum","force_count_persons_with":"sum","weapon_count_persons_with":"sum","trr_count_persons_with":"sum","dept_complaint_count_persons_with":"sum","dept_complaint_cleaned_count_persons_with":"sum","civ_complaint_count_persons_with":"sum","civ_complaint_force_persons_with":"sum","civ_complaint_detain_persons_with":"sum","civ_complaint_hate_persons_with":"sum","civ_complaint_crime_persons_with":"sum"}).reset_index()

for col in col_names:
    unit_all[col+"_persons_with_p"]=unit_all[col+"_persons_with"]/unit_all["UID"]

unit_all[["unit_description","UID","force_count"]].to_json("departments.json",orient="records")

unit_total=df[df.unit_description!="RECRUIT TRAINING SECTION"].groupby("one").agg({"UID":"count","force_count":"sum","weapon_count":"sum","force_count_persons_with":"sum","weapon_count_persons_with":"sum"}).reset_index()
unit_total.to_json("total_force.json",orient="records")



In [80]:
df['appointed_year']=df['appointed_date'].apply(lambda x: str(x)[0:4])
df['appointed_year']=df['appointed_year'].astype("int") 

df["years_in_2016"]=2017-df['appointed_year'].astype(int)
df["years_in_2016"]=np.where(df["years_in_2016"]==1017,-1,df["years_in_2016"])

df["full_name"]=df['first_name']+" "
df["full_name"]=np.where(df["middle_initial"].isna(),df["full_name"],df["full_name"]+df['middle_initial']+". ")
df["full_name"]=np.where(df["middle_initial2"].isna(),df["full_name"],df["full_name"]+df['middle_initial2']+". ")
df["full_name"]=df["full_name"]+df['last_name']
df["full_name"]=np.where(df["suffix_name"].isna(),df["full_name"],df["full_name"]+" "+df['suffix_name'])

In [81]:
cols_to_keep=['gender', 'race', 'birth_year', 'current_age','appointed_date', 'current_rank', 'unit_description', 'first_name', 'last_name', 'middle_initial', 'suffix_name','UID','dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'dept_complaint_desc', 'civ_complaint_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'force_count','weapon_count','trr_count', 'action_force_desc', 'action_weapon_desc']
cols_to_keep=['UID','full_name','last_name','birth_year','appointed_date', 'current_rank', 'unit_description','dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'dept_complaint_cleaned_count_desc', 'civ_complaint_count_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'force_count','weapon_count','trr_count', 'force_count_desc', 'weapon_count_desc','years_in_2016']

cols_to_keep_int=['dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'force_count','weapon_count','trr_count','years_in_2016']
cols_to_keep_str=['full_name','last_name','birth_year','appointed_date', 'current_rank', 'unit_description']
cols_to_keep_list=['dept_complaint_cleaned_count_desc', 'civ_complaint_count_desc','civ_complaint_force_desc', 'civ_complaint_detain_desc','civ_complaint_hate_desc', 'civ_complaint_crime_desc', 'force_count_desc', 'weapon_count_desc']


for col in cols_to_keep_int:
    df[col]=np.where(df[col].isna(),0,df[col])
for col in cols_to_keep_str:
    df[col]=np.where(df[col].isna(),"",df[col])

def func(x):
    counts,types = x
    text=""
    counts=counts.tolist()
    types=types.tolist()

    for i in range(len(counts)):
        text=text+"<br/>"+str(counts[len(counts)-1-i])+"x: "+types[len(counts)-1-i].capitalize()
    return text

df["empty"]=[list() for x in range(len(df.index))]

for col in cols_to_keep_list:
    df[col]=np.where(df[col].isna(),df["empty"],df[col])
    df[col+"_types"]=df[col].apply(lambda x: np.array(list(set(x)))[np.array([x.count(y) for y in set(x)]).argsort()])
    df[col+"_count"]=df[col].apply(lambda x: np.array([x.count(y) for y in set(x)])[np.array([x.count(y) for y in set(x)]).argsort()])
    df[col+"_listx"]=list(zip(df[col+"_count"],df[col+"_types"]))
    df[col+"_list"]=df[col+"_listx"].apply(lambda x: func(x))

df["full_name"]=df["full_name"].apply(lambda x: x.title())
df["current_rank"]=df["current_rank"].apply(lambda x: x.title())

cols_to_keep=['UID','full_name','last_name','birth_year','appointed_date', 'current_rank', 'unit_description','dept_complaint_cleaned_count', 'civ_complaint_count','civ_complaint_force', 'civ_complaint_detain', 'civ_complaint_hate','civ_complaint_crime', 'force_count','weapon_count','years_in_2016',
'dept_complaint_cleaned_count_desc_list', 'civ_complaint_count_desc_list','civ_complaint_force_desc_list', 'civ_complaint_detain_desc_list','civ_complaint_hate_desc_list', 'civ_complaint_crime_desc_list', 'force_count_desc_list', 'weapon_count_desc_list']

df[cols_to_keep].to_json("officers_2016.json",orient="records")


OSError: [Errno 22] Invalid argument: 'officers_2016.json'

# Update Department Json

Add officer data as list per department. Export as departments_officers.json

In [ ]:
import json

with open("departments.json") as file:
    data = json.load(file)

with open("officers_2016.json") as file:
    officers = json.load(file)

for d in data:
    d["officers"]=[]
    temp=[]
    for officer in officers:
        if officer["unit_description"]==d["unit_description"]:
            d["officers"].append(officer)

with open("departments_officers.json", "w") as outfile:
    json.dump(data, outfile)


# Tree Hierarchy

Reformat list of departments in hierarchical form under department categories. Bucket small departments as other. Export as departments_tree.json

In [ ]:
districts=pd.read_csv("raw_data/Districts.csv")
districts["District_No_str"]=districts["District_No"].astype(str)
districts["District_Id"]=np.where(districts["District_No"]>9,"DISTRICT 0"+districts["District_No_str"],"DISTRICT 00"+districts["District_No_str"])
districts["District_Title"]=np.where(districts["District_No"]>9,"0"+districts["District_No_str"]+": "+districts["District_Name"],"00"+districts["District_No_str"]+": "+districts["District_Name"])


In [ ]:
#replicating the groups 
tree=df[["unit_cat","unit_description"]].value_counts().reset_index()
tree.columns=["unit_cat","unit_description","count"]
tree2=df[["unit_cat"]].value_counts().reset_index()
tree2.columns=["unit_cat","count"]

colors=["#a63603","#e6550d","#fd8d3c","#fdae6b","#fdd0a2","#feedde","#D3D3D3"]
cats=df["unit_cat"].unique()

tree_dict={}
tree_dict["name"]="FULL ROSTER"
tree_dict["children"]=[]

for color_index, cat in enumerate(["DISTRICT","DETECTIVE","TRAINING","TRANSPORT","GANG ENFORCEMENT","OTHER"]):
    child={}
    child["name"]=cat
    child["color"]=colors[color_index]
    child["children"]=[]
    
    other_grandchild={}
    other_grandchild["name"]="OTHER"
    other_grandchild["color"]=colors[color_index]
    other_grandchild["children"]=[]

    hold=tree[tree.unit_cat==cat]
    for index in range(hold.shape[0]):
        grandchild={}
        if hold.iloc[index,1]=="":
            grandchild["name"]=hold.iloc[index,1]="UNKNOWN: LIKELY UNDERCOVER"
        if cat=="DISTRICT":
            hold2=districts[districts["District_Id"]==hold.iloc[index,1]]
            if hold2.shape[0]>0:                
                #print(hold2["District_Title"])
                grandchild["name"]=hold2.iloc[0]["District_Title"]
            else:
                grandchild["name"]=hold.iloc[index,1]
        else:
            grandchild["name"]=hold.iloc[index,1]

        grandchild["value"]=int(hold.iloc[index,2])
        grandchild["percent_value"]=hold.iloc[index,2]/hold["count"].sum()
        grandchild["color"]=colors[color_index]
        
        if grandchild["percent_value"]<0.02:
            other_grandchild["children"].append(grandchild)
        else:
            child["children"].append(grandchild)

    if len(other_grandchild["children"])==0:
        pass
    elif len(other_grandchild["children"])==1:
        child["children"].append(other_grandchild["children"][0])
    else:
        child["children"].append(other_grandchild)
    tree_dict["children"].append(child)
with open("departments_tree.json", "w") as outfile:
    json.dump(tree_dict,outfile)


# Percentage with Complaint or TRR Incident

Remove officers in Recruit Training Section. Format pie size variables. Export as percent_cat.json

In [ ]:
complaint_desc={"dept_complaint_cleaned_count_persons_with":"Severe Departmental Complaint","civ_complaint_count_persons_with":"Any Civilian Complaint","civ_complaint_force_persons_with":"Use of Force / Weapon","civ_complaint_detain_persons_with":"Handling of Arrest / Detainment","civ_complaint_hate_persons_with":"Bigotry / Verbal Abuse","civ_complaint_crime_persons_with":"Criminal Activity"}
cats=df["unit_cat"].unique()

temp=df[df.unit_description!="RECRUIT TRAINING SECTION"].groupby("one").agg({"UID":"count","force_count_persons_with":"sum","weapon_count_persons_with":"sum","trr_count_persons_with":"sum","dept_complaint_count_persons_with":"sum","dept_complaint_cleaned_count_persons_with":"sum","civ_complaint_count_persons_with":"sum","civ_complaint_force_persons_with":"sum","civ_complaint_detain_persons_with":"sum","civ_complaint_hate_persons_with":"sum","civ_complaint_crime_persons_with":"sum"}).reset_index()

#for all categories
pie_dict={}
pies=[]
for col in complaint_desc.keys():
    pie={}
    pie["name"]=complaint_desc[col]
    pie["denom"]=int(temp["UID"][0])
    yes={"Answer":"Yes","Totals":int(temp[col][0]),"Color":0}
    no={"Answer":"No","Totals":pie["denom"]-int(temp[col][0]),"Color":1}
    pie["Values"]=[yes,no]
    pies.append(pie)
pie_dict["ALL"]=pies

with open("percent_cat.json", "w") as outfile:
    json.dump(pie_dict,outfile)


In [86]:
df.force_weapon_count=df.force_count+df.weapon_count
max(df.force_weapon_count)

C:\Users\Olivia_Pinney\AppData\Local\Temp\ipykernel_29252\1499928351.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.force_weapon_count=df.force_count+df.weapon_count


222.0